<h2><center> Homework №2 </center></h2>

<h3>Contents</h3>
        <li><a href="#1">Environment Setup</a></li>
        <li><a href="#2">Task 1</a></li>
        <li><a href="#3">Task 2</a></li>
        <li><a href="#4">Task 3</a></li>

In [1]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geopandas.tools import sjoin
import folium
from folium.plugins import HeatMap, HeatMapWithTime

In [2]:
def embed_map(m):
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

<a name="1"></a>
<h3>Environment Setup</h3>

In [3]:
import os
import sys

os.environ["SPARK_HOME"]="/home/ubuntu/BigData/spark"
os.environ["PYSPARK_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

In [4]:
import pyspark
from pyspark.sql import SparkSession, Row

In [5]:
conf = pyspark.SparkConf() \
        .set("spark.driver.memory", "2g") \
        .set("spark.executor.memory", "1g") \
        .set("spark.executor.core", "2") \
        .setAppName("HW2") \
        .setMaster("local[4]")

In [6]:
spark = SparkSession \
    .builder \
    .config(conf=conf) \
    .getOrCreate()

In [7]:
spark

In [8]:
import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, 
    StructField, 
    StringType, 
    IntegerType, 
    DoubleType,
    FloatType
)
from math import cos, asin, sqrt, sin

In [9]:
# spark.stop()

<a name="2"></a>
<h3>Task 1</h3>

<h4>Task 1.1</h4>

In [18]:
BASE_PATH = "/home/ubuntu/BigData/datasets"
trips_data_path = "file://{}/201902-citibike-tripdata.csv".format(BASE_PATH)

In [19]:
df_trips = spark.read.load(trips_data_path, 
                           format="csv", 
                           header="true", 
                           inferSchema="true",
                           sep=",")

df_trips.show(5, truncate=True)

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|           starttime|            stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|         219|2019-02-01 00:00:...|2019-02-01 00:03:...|            3494|E 115 St & Lexing...|             40.797911|               -73.9423|          3501|E 118 St & Madiso...|          40.8014866|          -73.9442507| 33450|Su

Получаем количество начала поездок

In [20]:
df_stations_start_count = df_trips.select(F.col("start station id").alias("id"))\
                            .groupBy("id")\
                            .agg(F.count("id").alias("start_count"))
df_stations_start_count.show(5)

+----+-----------+
|  id|start_count|
+----+-----------+
| 296|       1795|
|3606|        215|
| 467|       1276|
|3414|        656|
|3368|        745|
+----+-----------+
only showing top 5 rows



Получаем количество завершения поездок

In [21]:
df_stations_end_count = df_trips.select(F.col("end station id").alias("id"))\
                            .groupBy("id")\
                            .agg(F.count("id").alias("end_count"))
df_stations_end_count.show(5)

+----+---------+
|  id|end_count|
+----+---------+
| 467|     1312|
| 296|     1825|
|3414|      681|
|3606|      224|
|3441|      268|
+----+---------+
only showing top 5 rows



Получаем список всех станций

In [22]:
df_stations_start = df_trips.select(F.col("start station id").alias("id"), 
                              F.col("start station latitude").alias("lat"), 
                              F.col("start station longitude").alias("lng"))\
                            .distinct()

df_stations_end = df_trips.select(F.col("end station id").alias("id"), 
                              F.col("end station latitude").alias("lat"), 
                              F.col("end station longitude").alias("lng"))\
                            .distinct()

df_stations = df_stations_start.union(df_stations_end).distinct()
print("Number of stations:", df_stations.count())

df_stations.show(5)

Number of stations: 780
+----+-----------+------------+
|  id|        lat|         lng|
+----+-----------+------------+
| 379|  40.749156|    -73.9916|
|3495| 40.7945663| -73.9362541|
|3490|  40.796879|  -73.937261|
| 340|40.71269042|-73.98776323|
|3391| 40.7892529|-73.93956237|
+----+-----------+------------+
only showing top 5 rows



Результат

In [23]:
df_stations_count = df_stations.join(df_stations_start_count, on="Id", how="left")\
                                .join(df_stations_end_count, on="Id", how="left")

df_stations_count.show()

+----+------------------+------------------+-----------+---------+
|  id|               lat|               lng|start_count|end_count|
+----+------------------+------------------+-----------+---------+
| 296|       40.71413089|       -73.9970468|       1795|     1825|
|3414|40.680944723477296|-73.97567331790923|        656|      681|
|3606|          40.74252|        -73.948852|        215|      224|
| 467|       40.68312489|      -73.97895137|       1276|     1312|
|3368|        40.6728155|      -73.98352355|        745|      769|
|3441|         40.752957|         -74.00264|        263|      268|
|3517|        40.7711528|       -73.9170074|        299|      301|
|3526|        40.7747878|       -73.9125551|        604|      606|
|3121|       40.74524768|      -73.94733276|        291|      295|
|3249|  40.6803560840434| -73.9476791024208|        341|      340|
|3312|        40.7817212|         -73.94594|        788|      791|
| 447|       40.76370739|       -73.9851615|       2512|     2

In [16]:
# df_stations_count.createOrReplaceTempView("station_without_start_trips")
# df_grade = spark.sql("SELECT * FROM station_without_start_trips WHERE id = 3639")
# df_grade.show()

+----+----------+----------+-----------+---------+
|  id|       lat|       lng|start_count|end_count|
+----+----------+----------+-----------+---------+
|3639|40.7192517|-74.034234|       null|        1|
+----+----------+----------+-----------+---------+



<h4> Task 1.2, 1.3 </h4>

In [24]:
borough_data_path = BASE_PATH + "/NYC Taxi Zones.geojson"


with open(borough_data_path) as f:
    zones_geojson = json.load(f)
    
    
column_name_list = [key for key, _ in zones_geojson["features"][0]["properties"].items()]
column_name_list += ["geometry"]

In [25]:
def create_zone_rows(features):
    for item in features:
        row = list()
        for key, value in item["properties"].items():
            row.append(value)        
        polygons = list()
        for polygon in item["geometry"]["coordinates"]:
            polygons.append(Polygon(polygon[0]))
        row.append(MultiPolygon(polygons=polygons))
        yield row

In [27]:
df_zones_pn = pd.DataFrame(create_zone_rows(zones_geojson["features"]), columns=column_name_list)
gdf_zones = gpd.GeoDataFrame(df_zones_pn, geometry=df_zones_pn["geometry"])
gdf_zones.head(5)

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,"MULTIPOLYGON (((-74.18445 40.69500, -74.18449 ..."
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,"MULTIPOLYGON (((-73.84793 40.87134, -73.84725 ..."
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,"MULTIPOLYGON (((-73.97177 40.72582, -73.97179 ..."
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,"MULTIPOLYGON (((-74.17422 40.56257, -74.17349 ..."


In [28]:
bc_zones = spark.sparkContext.broadcast(gdf_zones)
bc_zones

In [29]:
def zone_contains_v2_df(rows):
    points = list()
    for row in rows: 
        points.append([row["lng"], row["lat"], row["start_count"], row["end_count"]])
    if len(points) == 0:
        return list()
    df_points_pn = pd.DataFrame(points, columns=["lng", "lat", "start_count", "end_count"])
    gdf_points = gpd.GeoDataFrame(df_points_pn, 
                                  geometry=gpd.points_from_xy(df_points_pn["lng"],
                                                              df_points_pn["lat"]))
    for index, item in sjoin(gdf_points, bc_zones.value, how="left").iterrows():
        yield Row(id=item["location_id"], zone=item["zone"], start_c = item["start_count"], end_c = item["end_count"])

Количество поездок для каждого квартала, отсортированный по убыванию количества начала поездок

In [30]:
df_zones_count = df_stations_count.rdd\
    .mapPartitions(zone_contains_v2_df).toDF()\
    .groupBy("id", "zone")\
    .agg(F.sum("start_c").alias("start_count"), F.sum("end_c").alias("end_count"))\
    .sort("start_count", ascending = False)

df_zones_count.show(5)

+---+--------------------+-----------+---------+
| id|                zone|start_count|end_count|
+---+--------------------+-----------+---------+
| 79|        East Village|      43333|    43170|
| 68|        East Chelsea|      38967|    39395|
|170|         Murray Hill|      32572|    32578|
|234|            Union Sq|      30527|    31464|
|113|Greenwich Village...|      26485|    27054|
+---+--------------------+-----------+---------+
only showing top 5 rows



Количество поездок для каждого квартала, отсортированный по убыванию количества завершения поездок

In [31]:
df_zones_count.sort("end_count", ascending = False).show(5)

+---+--------------------+-----------+---------+
| id|                zone|start_count|end_count|
+---+--------------------+-----------+---------+
| 79|        East Village|      43333|    43170|
| 68|        East Chelsea|      38967|    39395|
|170|         Murray Hill|      32572|    32578|
|234|            Union Sq|      30527|    31464|
|113|Greenwich Village...|      26485|    27054|
+---+--------------------+-----------+---------+
only showing top 5 rows



<h4> Task 1.4 </h4>

In [32]:
df_zones_count_pn = df_zones_count.toPandas()
df_zones_count_pn.head(5)

,id,zone,start_count,end_count
0,79,East Village,43333.0,43170
1,68,East Chelsea,38967.0,39395
2,170,Murray Hill,32572.0,32578
3,234,Union Sq,30527.0,31464
4,113,Greenwich Village North,26485.0,27054


Картограмма начала поездок

In [33]:
m = folium.Map()

folium.Choropleth(
    geo_data=zones_geojson,
    data=df_zones_count_pn,
    columns=["id", "start_count"], #2 столбца
    name="Number of start trips",
    legend_name="Number of trips",
    key_on="feature.properties.location_id",
    highlight=True,
    nan_fill_color="grey",
    nan_fill_opacity=0.1,
    fill_color="PuRd",
    fill_opacity=0.7,
    line_opacity=0.2,
).add_to(m)
m.fit_bounds(m.get_bounds())
embed_map(m)

Картограмма завершения поездок

In [27]:
m2 = folium.Map()

folium.Choropleth(
    geo_data=zones_geojson,
    data=df_zones_count_pn,
    columns=["id", "end_count"], #2 столбца
    name="Number of end trips",
    legend_name="Number of trips",
    key_on="feature.properties.location_id",
    highlight=True,
    nan_fill_color="grey",
    nan_fill_opacity=0.1,
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
).add_to(m2)
m2.fit_bounds(m.get_bounds())
embed_map(m2)

<a name="3"></a>
<h3>Task 2</h3>

In [28]:
df_trips

DataFrame[tripduration: int, starttime: timestamp, stoptime: timestamp, start station id: string, start station name: string, start station latitude: double, start station longitude: double, end station id: string, end station name: string, end station latitude: double, end station longitude: double, bikeid: int, usertype: string, birth year: int, gender: int]

In [29]:
filter_expr = (F.col("start_lat") != F.col("end_lat")) & (F.col("start_lng") != F.col("end_lng"))

In [39]:
df_trips_coord = df_trips.select(F.col("start station id").alias("start_id"),
                              F.col("start station latitude").alias("start_lat"), 
                              F.col("start station longitude").alias("start_lng"),
                              F.col("end station id").alias("end_id"),
                              F.col("end station latitude").alias("end_lat"), 
                              F.col("end station longitude").alias("end_lng"))\
                        .where(filter_expr)

df_trips_coord.show(5)

+--------+-----------------+-----------------+------+-----------------+------------------+
|start_id|        start_lat|        start_lng|end_id|          end_lat|           end_lng|
+--------+-----------------+-----------------+------+-----------------+------------------+
|    3494|        40.797911|         -73.9423|  3501|       40.8014866|       -73.9442507|
|     438|      40.72779126|     -73.98564945|   236|       40.7284186|      -73.98713956|
|    3571|        40.676368|       -73.952918|  3549|        40.678045|        -73.962408|
|     167|       40.7489006|     -73.97604882|   477|      40.75640548|       -73.9900262|
|    3458|40.76309387270797|-73.9783501625061|  3443|40.76132983124814|-73.97982001304626|
+--------+-----------------+-----------------+------+-----------------+------------------+
only showing top 5 rows



In [40]:
def distance(data):
    lat1 = data[0]
    lng1 = data[1]
    lat2 = data[2]
    lng2 = data[3]
    p = 0.017453292519943295
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lng2 - lng1) * p)) / 2
    return 12742 * asin(sqrt(a)) * 1000
    
    
distance_udf = F.udf(distance, DoubleType())

In [41]:
df_trips_dist = df_trips_coord.withColumn("trip_dist", distance_udf(F.array('start_lat', 'start_lng', 'end_lat', 'end_lng')))
df_trips_dist.show(5)

+--------+-----------------+-----------------+------+-----------------+------------------+------------------+
|start_id|        start_lat|        start_lng|end_id|          end_lat|           end_lng|         trip_dist|
+--------+-----------------+-----------------+------+-----------------+------------------+------------------+
|    3494|        40.797911|         -73.9423|  3501|       40.8014866|       -73.9442507|430.16041257402225|
|     438|      40.72779126|     -73.98564945|   236|       40.7284186|      -73.98713956|143.63994354824555|
|    3571|        40.676368|       -73.952918|  3549|        40.678045|        -73.962408| 821.7251364836256|
|     167|       40.7489006|     -73.97604882|   477|      40.75640548|       -73.9900262|1443.1216859041303|
|    3458|40.76309387270797|-73.9783501625061|  3443|40.76132983124814|-73.97982001304626|231.94966562338428|
+--------+-----------------+-----------------+------+-----------------+------------------+------------------+
only showi

Медиана:

In [42]:
median = df_trips_dist.approxQuantile("trip_dist", [0.5], 0)[0]
median

1262.874571344554

In [43]:
a = spark.createDataFrame([('median', median)], ['summarry', 'trip_dist'])
a.show()

+--------+-----------------+
|summarry|        trip_dist|
+--------+-----------------+
|  median|1262.874571344554|
+--------+-----------------+



Все показатели:

In [44]:
df_trips_dist.describe("trip_dist").union(a).show()

+-------+------------------+
|summary|         trip_dist|
+-------+------------------+
|  count|            928612|
|   mean|1640.3742164463074|
| stddev|1288.6756355029727|
|    min| 36.73587142648394|
|    max|15326.431486590483|
| median| 1262.874571344554|
+-------+------------------+



<a name="4"></a>
<h3>Task 3</h3>

In [36]:
df_trips

DataFrame[tripduration: int, starttime: timestamp, stoptime: timestamp, start station id: string, start station name: string, start station latitude: double, start station longitude: double, end station id: string, end station name: string, end station latitude: double, end station longitude: double, bikeid: int, usertype: string, birth year: int, gender: int]

In [37]:
df_trips_start_time = df_trips.select(F.col("start station id").alias("station_id"), 
                                    F.to_date(F.col("starttime")).alias("start_day"))\
                                .distinct()\
                                .groupBy("station_id")\
                                .agg(F.count("start_day").alias("start_day_count"))

df_trips_start_time.show(5)

+----------+---------------+
|station_id|start_day_count|
+----------+---------------+
|       296|             28|
|      3414|             28|
|      3606|             28|
|       467|             28|
|      3441|             28|
+----------+---------------+
only showing top 5 rows



In [38]:
df_trips_stop_time = df_trips.select(F.col("end station id").alias("station_id"), 
                                  F.to_date(F.col("stoptime")).alias("stop_day"))\
                             .distinct()\
                             .groupBy("station_id")\
                             .agg(F.count("stop_day").alias("stop_day_count"))

df_trips_stop_time.show(5)

+----------+--------------+
|station_id|stop_day_count|
+----------+--------------+
|       296|            29|
|      3414|            28|
|      3606|            28|
|       467|            29|
|      3441|            28|
+----------+--------------+
only showing top 5 rows



In [110]:
df_stations_count.show(5)

+----+------------------+------------------+-----------+---------+
|  id|               lat|               lng|start_count|end_count|
+----+------------------+------------------+-----------+---------+
| 296|       40.71413089|       -73.9970468|       1795|     1825|
|3414|40.680944723477296|-73.97567331790923|        656|      681|
|3606|          40.74252|        -73.948852|        215|      224|
| 467|       40.68312489|      -73.97895137|       1276|     1312|
|3368|        40.6728155|      -73.98352355|        745|      769|
+----+------------------+------------------+-----------+---------+
only showing top 5 rows



In [121]:
avg_by_day = df_stations_count.select(F.col("id").alias("station_id"), 
                                 F.col("start_count"), F.col("end_count"))\
                            .join(df_trips_start_time, on="station_id", how="left")\
                            .join(df_trips_stop_time, on="station_id", how="left")\
                            .withColumn("avg_start_count_day", F.round(F.col("start_count")/F.col("start_day_count")))\
                            .withColumn("avg_stop_count_day", F.round(F.col("end_count")/F.col("stop_day_count")))\
                            .select((F.col("station_id")),
                            (F.col("avg_start_count_day").alias("avg_count_start")),
                            (F.col("avg_stop_count_day").alias("avg_count_stop")))

avg_by_day.show(20)

+----------+---------------+--------------+
|station_id|avg_count_start|avg_count_stop|
+----------+---------------+--------------+
|       296|           64.0|          63.0|
|      3414|           23.0|          24.0|
|      3606|            8.0|           8.0|
|       467|           46.0|          45.0|
|      3368|           27.0|          27.0|
|      3441|            9.0|          10.0|
|      3517|           11.0|          11.0|
|      3526|           22.0|          22.0|
|      3121|           10.0|          11.0|
|      3249|           12.0|          12.0|
|      3312|           28.0|          28.0|
|       447|           90.0|          89.0|
|      3057|           17.0|          16.0|
|       307|          108.0|         107.0|
|      3167|          101.0|          96.0|
|      3491|           14.0|          14.0|
|      3179|            9.0|           9.0|
|      3241|           16.0|          15.0|
|      3299|            9.0|           9.0|
|       334|          104.0|    

In [122]:
# spark.stop()